In [28]:
import sys
import os
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime

sys.path.append(os.path.abspath("D:\\burtm\\Visual_studio_code\\PD_related_projects"))

In [29]:
selected_model = "trocr-2transformer-blocks"
selected_transform = "trocr-small-stage1"
N_max=282
use_patches=True
pretrained=True
depth=-1
hugging=True
num_epochs=100
batch_size=32
learning_rate=0.001
input_filename='icdar_train_df_cc_5patches_perName.csv' #'representations.h5'
criterion_name="CrossEntropyLoss"
criterion = training_utils.get_criterion(criterion_name)
optimizer_name = "Adam"
num_classes = 2  # Change this to match your dataset
early_stopping=10
scheduler_name = 'no_scheduling'#CosineAnnealingLR'
checkpoint_path = "D:\\burtm\Visual_studio_code\PD_related_projects\checkpoints\\"
models_path = "D:\\burtm\Visual_studio_code\PD_related_projects\outputs\models\\"

In [30]:
# Example training metadata
training_metadata = {
    "type_of_approach": "training transformer on top of trocr extracted features",
    "type_of_approach_sigla": "accelerated_fine_tuning_trocr",
    "model_name": selected_model,
    "transform_name": selected_transform,
    "epochs": num_epochs,
    "batch_size": batch_size,
    "learning_rate": learning_rate,
    "optimizer": optimizer_name,
    "pretrained": pretrained,
    "depth": depth,
    "use_patches": use_patches,
    "input_filename": input_filename,
    "num_classes": num_classes,
    "criterion_name": criterion_name,
    "early_stopping": early_stopping,
    "N_max": N_max,
    "scheduler_name": scheduler_name,
}

In [75]:
transform=u_transforms.get_transform(selected_transform,use_patches=use_patches)
train_dataloader,val_dataloader=data_loading.get_dataloaders(transform, batch_size=16, N_max=282, 
                                                file_name=input_filename,hugging=hugging,h5=True)



loaded train_df from: D:\burtm\Visual_studio_code\PD_related_projects\outputs\preprocessed_data\icdar_train_df_cc_5patches_perName.csv


In [32]:
# Get one batch from the dataloader
batch = next(iter(train_dataloader))

# Print the shape of the 'image' element in the batch
print(batch['features'][0].shape)

torch.Size([578, 384])


In [80]:
model=model_utils.get_model(selected_model, pretrained=pretrained, num_classes=num_classes,
                            input_size=384, hidden_sizes=[128])

# Define loss function and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device is: ",device)
model = model.to(device)

Config of the encoder: <class 'transformers.models.deit.modeling_deit.DeiTModel'> is overwritten by shared encoder config: DeiTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 384,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "model_type": "deit",
  "num_attention_heads": 6,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.48.2"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 384,
  "d_model": 256,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 1024,
  "decode

Device is:  cuda


In [50]:
print(model)

FullClassifier(
  (encoder): TruncatedDeiT(
    (transformer): ModuleList(
      (0-1): 2 x DeiTLayer(
        (attention): DeiTSdpaAttention(
          (attention): DeiTSdpaSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): DeiTSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): DeiTIntermediate(
          (dense): Linear(in_features=384, out_features=1536, bias=True)
          (intermediate_act_fn): GELUActivation()
        )
        (output): DeiTOutput(
          (dense): Linear(in_features=1536, out_features=384, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
  

In [72]:
model.forward??

Signature: model.forward(x)
Docstring:
Define the computation performed at every call.

Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.
Source:   
    def forward(self, x):
        outputs = self.encoder(x)
        #cls_token_output = outputs.last_hidden_state[:, 0, :]  # Extract the CLS token
        if self.strategy == 'cls':
            print(outputs.shape)
            output = outputs[:, 0, :]
        elif self.strategy == 'mean':
            output = outputs.mean(dim=1)
        logits = self.classification_head(output)
        return logits
File:      d:\burtm\visual_studio_code\pd_related_projects\utils\model_utils.py
Type:      method

In [81]:
# Freeze all layers but the last n
num_trainable_layers = model_utils.get_trainable_layers(selected_model,depth=depth) 
model = training_utils.fine_tune_last_n_layers(model, num_trainable_layers)
optimizer = training_utils.get_optimizer(model, optimizer_name, lr=learning_rate)
scheduler = training_utils.get_scheduler(optimizer, scheduler_name, T_max=num_epochs)

Total Layers: 40
Total Parameters: 3,747,074
Trainable Parameters after freezing: 0


Unfreezing the following layers:
Linear(in_features=384, out_features=384, bias=True)
Linear(in_features=384, out_features=384, bias=True)
Linear(in_features=384, out_features=384, bias=True)
Dropout(p=0.0, inplace=False)
Linear(in_features=384, out_features=384, bias=True)
Dropout(p=0.0, inplace=False)
Linear(in_features=384, out_features=1536, bias=True)
GELUActivation()
Linear(in_features=1536, out_features=384, bias=True)
Dropout(p=0.0, inplace=False)
LayerNorm((384,), eps=1e-12, elementwise_affine=True)
LayerNorm((384,), eps=1e-12, elementwise_affine=True)
Linear(in_features=384, out_features=384, bias=True)
Linear(in_features=384, out_features=384, bias=True)
Linear(in_features=384, out_features=384, bias=True)
Dropout(p=0.0, inplace=False)
Linear(in_features=384, out_features=384, bias=True)
Dropout(p=0.0, inplace=False)
Linear(in_features=384, out_features=1536, bias=True)
GELUActivation()
Lin

In [71]:
# Move the batch features to the same device as the model
features = batch['features'].to(device)
#print(features[0].shape)

# Apply the model to the batch
outputs = model(features)

# Print the outputs
#print(outputs)

torch.Size([16, 384])


IndexError: too many indices for tensor of dimension 2

In [ ]:
start_time=datetime.now()
model,train_losses,val_losses = training_utils.train_model(model, train_dataloader, val_dataloader, criterion, optimizer, device, 
                                            num_epochs=num_epochs, 
                                            checkpoint_path=checkpoint_path,
                                            early_stopping_patience=early_stopping, scheduler=scheduler, data_type='features')
end_time=datetime.now()

Epoch 1/100: 100%|██████████| 317/317 [10:12<00:00,  1.93s/it]


Epoch 1, Train Loss: 0.6988, Train Acc: 0.5091
Epoch 1, Val Loss: 0.7025, Val Acc: 0.4483
Checkpoint saved: D:\burtm\Visual_studio_code\PD_related_projects\checkpoints\best_checkpoint.pth


Epoch 2/100: 100%|██████████| 317/317 [09:20<00:00,  1.77s/it]


Epoch 2, Train Loss: 0.6928, Train Acc: 0.5202
Epoch 2, Val Loss: 0.6939, Val Acc: 0.4655
Checkpoint saved: D:\burtm\Visual_studio_code\PD_related_projects\checkpoints\best_checkpoint.pth


Epoch 3/100: 100%|██████████| 317/317 [08:18<00:00,  1.57s/it]


...

Epoch 5/100: 100%|██████████| 317/317 [10:14<00:00,  1.94s/it]


Epoch 5, Train Loss: 0.6929, Train Acc: 0.5138
Epoch 5, Val Loss: 0.6955, Val Acc: 0.4483
Checkpoint saved: D:\burtm\Visual_studio_code\PD_related_projects\checkpoints\last_checkpoint.pth


Epoch 6/100: 100%|██████████| 317/317 [08:25<00:00,  1.59s/it]


Epoch 6, Train Loss: 0.6929, Train Acc: 0.5138
Epoch 6, Val Loss: 0.6959, Val Acc: 0.4483
Checkpoint saved: D:\burtm\Visual_studio_code\PD_related_projects\checkpoints\last_checkpoint.pth


Epoch 7/100:   5%|▌         | 16/317 [00:33<08:26,  1.68s/it]

In [ ]:
#get the best_checkpoint.pth and add the training metadata to it
checkpoint = torch.load(checkpoint_path+'best_checkpoint.pth')
checkpoint['training_metadata'] = training_metadata
val_accuracy= checkpoint['val_acc']
# Save the updated checkpoint with metadata
torch.save(checkpoint, checkpoint_path)

#do the same for the last checkpoint
checkpoint = torch.load(checkpoint_path+'last_checkpoint.pth')
checkpoint['training_metadata'] = training_metadata
# Save the updated checkpoint with metadata
torch.save(checkpoint, checkpoint_path)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
torch.save(checkpoint, f"{models_path}\{training_metadata['type_of_approach_sigla']}_ValAcc{val_accuracy}_{timestamp}.pth")

# easy access

In [79]:
def reload_modules():
    import importlib
    import utils.data_loading as data_loading
    import utils.visualization as visualization
    import utils.dataframes as dataframes
    import utils.utils_transforms as u_transforms
    import utils.training_utils as training_utils
    import utils.model_utils as model_utils
    

    importlib.reload(data_loading)
    importlib.reload(visualization)
    importlib.reload(dataframes)
    importlib.reload(u_transforms)
    importlib.reload(model_utils)
    importlib.reload(training_utils)

    return data_loading, visualization, dataframes, u_transforms, training_utils, model_utils
data_loading, visualization, dataframes, u_transforms, training_utils, model_utils = reload_modules()